In [57]:
from collections import namedtuple
import os
import urllib.request as request
from zipfile import ZipFile
import shutil

In [2]:
os.getcwd()

'/Users/vikaslakka/Desktop/FSDS/Computer_vision/projects/vehicle_detection_YOLO/research'

In [3]:
os.path.exists("dataset/dataset.zip")

False

In [4]:
## Change working directory
os.chdir("/Users/vikaslakka/Desktop/FSDS/Computer_vision/projects/vehicle_detection_YOLO")

In [86]:
## Create Dataingestion config
DataIngestionConfig= namedtuple("DataIngestionConfig",
                                [
                                    "root_dir",
                                    "source_url",
                                    "local_data_file",
                                    "unzip_dir",
                                    "zip_file_dir"
                                    "dataset_source"

                                ])

In [87]:
## Create data classes to consider from data ingestion Configs
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    zip_file_dir: Path
    dataset_source: Path

#### Unzipping the dataset available and storing to right folder

In [63]:
from vehicle_detection.constants import *
from vehicle_detection.utils.common import read_yaml, create_directories

In [88]:
class ConfiguraionManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH):
        """
        Creating artifact directories from config.yaml file
        """
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        ##Creating Directories
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """Creating directory for data ingestion from config.yaml file.

        Returns:
            DataIngestionConfig: Dataingestion data
        """
        config= self.config.data_ingestion
        ##Create directory that are required for data ingestion(taken from config.yaml)
        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir,
            zip_file_dir= config.zip_file_dir,
            dataset_source= config.dataset_source
        )
        return data_ingestion_config



### now create data set and download into respective folders

In [95]:
## Data Ingestion component
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config= config
    def _preprocess(self, zf: ZipFile, f:str, working_dir:str):
        """places all files into target folder
           ignores files which has size of zero

        Args:
            zf (ZipFile): Zip file
            f (str): file name with path
            working_dir (str): targer working directory
        """
        target_filepath= os.path.join(working_dir, f)
        dir_name= os.path.dirname(target_filepath)
        #print(f"filepath: {f}")
        
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)##Create file folder
        print(f"target file path: {target_filepath}")
        
        
        
        ##If the folder has size of zero then delete.
        if os.path.getsize(target_filepath)==0:
            os.remove(target_filepath)
    def copy_dataset_destination(self, source: Path, destination: Path):
        ###Copy dataset into Data ingestion folder
        shutil.copy2(source, destination)
        
    def unzip_and_clean(self):
        """unzip the dataset and clean the files 
           which has zero size
        """
        ##print(self.config.local_data_file)
        with ZipFile(file= self.config.local_data_file, mode="r") as zf:
            ##Getting the list of files available.
            list_of_files= zf.namelist()
            zf.extractall(self.config.unzip_dir)

            ##extracting files that has only filename dataset
            '''for f in list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)'''

### Try out Data ingestion

In [97]:
try:
    config= ConfiguraionManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config= data_ingestion_config)
    data_ingestion.copy_dataset_destination(data_ingestion_config.dataset_source, data_ingestion_config.unzip_dir)
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2023-09-26 09:32:29,770: INFO: common]: yaml file:configs/config.yaml loaded successfully
[2023-09-26 09:32:29,772: INFO: common]: yaml file:params.yaml loaded successfully
[2023-09-26 09:32:29,773: INFO: common]: Created directory at:artifacts
[2023-09-26 09:32:29,774: INFO: common]: Created directory at:artifacts/data_ingestion
